In [10]:
#Imports
from fileinput import filename
import random
import math
import matplotlib
from scipy.io import loadmat
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import sklearn.linear_model as ln
import sklearn.ensemble as es
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


from cross_validate import cross_validate_model

In [24]:
# Load only the numerical sensor data (assuming columns 4 onward are channels)
sensor_data = np.loadtxt(
    './datasets/CoST.csv',
    delimiter=',',
    skiprows=1,           # Skip the header row
)
print(f"Loaded shape: {sensor_data.shape}")



feat = sensor_data[:, 4:].astype(float)  # All rows, columns from index 4 onward
gnd_variant = sensor_data[:, 1].astype(int) - 1  # All rows, second column
gnd_gesture = sensor_data[:, 2].astype(int) - 1  # All rows, third column

print(np.unique(gnd_gesture))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feat, gnd_gesture.astype(int), test_size=0.2, random_state=42)


Loaded shape: (1496855, 68)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]


In [30]:


print("\n=== Training Final EBM Model ===")
best_model = ExplainableBoostingClassifier(    
    max_bins=64,              # down from 256/1024
    max_interaction_bins=16,  # lighter interaction binning
    interactions=5,           # or 5 if you really want some interactions
    outer_bags=4,             # fewer bagged models
    inner_bags=0,
    max_rounds=1000,           # not 5000+
    learning_rate=0.2,       # a bit higher to compensate fewer rounds
    validation_size=0.1,      # keep early stopping
    early_stopping_rounds=25,
    n_jobs=-1,                # use all cores
    random_state=0
)

best_model.fit(X_train, y_train)
print("Trained final EBM model with best hyperparameters.")


show(best_model.explain_global())


test_pred = best_model.predict(X_test)
y_true = y_test

print("Test Predictions:")
print(test_pred[:10])
print("True Values:")
print(y_true[:10])


# Calculate and print accuracy
accuracy = np.mean(test_pred == y_true)
print(f"Accuracy: {accuracy}")



=== Training Final EBM Model ===


/home/chasen/Projects/ml-project/.venv/lib/python3.13/site-packages/interpret/glassbox/_ebm/_ebm.py:1250: UserWarning:

For multiclass we cannot currently visualize pairs and they will be stripped from the global explanations. Set interactions=0 to generate a fully interpretable glassbox model.

Traceback (most recent call last):
  File "/home/chasen/Projects/ml-project/.venv/lib/python3.13/site-packages/joblib/externals/loky/backend/resource_tracker.py", line 326, in main
    registry[rtype][name] -= 1
    ~~~~~~~~~~~~~~~^^^^^^
KeyError: '/dev/shm/joblib_memmapping_folder_2147395_6a0383ec3814413db134a62d9d5b43ae_e9bdf06a55fa471e9c31803017c33756/2147395-140047201050768-8ebc1c234e8146ad8304fb72a551d5a1.pkl'


Trained final EBM model with best hyperparameters.


/home/chasen/Projects/ml-project/.venv/lib/python3.13/site-packages/interpret/glassbox/_ebm/_ebm.py:2204: UserWarning:

Dropping term feature_0017 & feature_0041 from explanation since we can't graph multinomial interactions.

/home/chasen/Projects/ml-project/.venv/lib/python3.13/site-packages/interpret/glassbox/_ebm/_ebm.py:2204: UserWarning:

Dropping term feature_0020 & feature_0035 from explanation since we can't graph multinomial interactions.

/home/chasen/Projects/ml-project/.venv/lib/python3.13/site-packages/interpret/glassbox/_ebm/_ebm.py:2204: UserWarning:

Dropping term feature_0027 & feature_0028 from explanation since we can't graph multinomial interactions.

/home/chasen/Projects/ml-project/.venv/lib/python3.13/site-packages/interpret/glassbox/_ebm/_ebm.py:2204: UserWarning:

Dropping term feature_0028 & feature_0035 from explanation since we can't graph multinomial interactions.

/home/chasen/Projects/ml-project/.venv/lib/python3.13/site-packages/interpret/glassbox/_ebm/

<!-- http://127.0.0.1:7916/140047196799312/ -->

Test Predictions:
[12  6  5  2 13  2  0  4  8 13]
True Values:
[12  6  5  2 13  2  0 10  2  5]
Accuracy: 0.4581405680576943


In [34]:

from sklearn.metrics import accuracy_score

# Load only the numerical sensor data (assuming columns 4 onward are channels)
sensor_data = np.loadtxt(
    "./datasets/CoST.csv",
    delimiter=",",
    skiprows=1,  # Skip the header row
)
print(f"Loaded CoST shape: {sensor_data.shape}")

# Columns:
# 0: subject
# 1: variant
# 2: gesture (class label)
# 3: frame index
# 4-67: ch1..ch64

feat = sensor_data[:, 4:]                      # (N_frames, 64)
gnd_variant = sensor_data[:, 1].astype(int)    # (N_frames,)
gnd_gesture = sensor_data[:, 2].astype(int)    # (N_frames,)

frame_column = sensor_data[:, 3].astype(int)   # (N_frames,) frame numbers

max_frame = np.max(frame_column)
print(f"Max frame length: {max_frame}")

# ----- Build sequences based on frame == 1 -----
sequence_change = []
for i, frame in enumerate(frame_column):
    if frame == 1:
        sequence_change.append(i)

print(f"Number of sequence starts: {len(sequence_change)}")

tensor_feat = np.split(feat, sequence_change[1:])
gnd_gesture_seq = gnd_gesture[sequence_change]   # gesture label per sequence
gnd_variant_seq = gnd_variant[sequence_change]   # variant label per sequence

print(f"Number of sequences after split: {len(tensor_feat)}")

# ----- Pad sequences to length max_frame -----
final_3d_array = np.zeros((len(tensor_feat), max_frame, 64), dtype=np.float32)

for i, seq in enumerate(tensor_feat):
    seq_len = seq.shape[0]
    final_3d_array[i, :seq_len, :] = seq

feat_seq = final_3d_array              # shape: (R, T_max, 64)
gnd_seq = (gnd_gesture_seq - 1).astype(int)   # zero-based labels

R, T_max, C_channels = feat_seq.shape
print(f"Final tensor shape: R={R}, T_max={T_max}, C={C_channels}")

# ---------------------------------------------------
# Hand-crafted time-series features (per sequence)
# ---------------------------------------------------

feat_mean = np.mean(feat_seq, axis=1)          # (R, 64)
feat_std = np.std(feat_seq, axis=1)            # (R, 64)
feat_max = np.max(feat_seq, axis=1)            # (R, 64)

feat_diff = np.diff(feat_seq, axis=1)          # (R, T_max-1, 64)
feat_mean_abs_diff = np.mean(np.abs(feat_diff), axis=1)  # (R, 64)

feat_accel = np.diff(feat_seq, axis=1, n=2)    # (R, T_max-2, 64)
feat_std_accel = np.std(feat_accel, axis=1)    # (R, 64)

mid_point = T_max // 2
feat_mean_half1 = np.mean(feat_seq[:, :mid_point, :], axis=1)  # (R, 64)
feat_mean_half2 = np.mean(feat_seq[:, mid_point:, :], axis=1)  # (R, 64)

feat_max_time = np.argmax(feat_seq, axis=1)    # (R, 64)
signs = np.sign(feat_seq)
sign_changes = np.diff(signs, axis=1) != 0
feat_zcr = np.sum(sign_changes, axis=1) / (T_max - 1)         # (R, 64)

feat_max_time_norm = feat_max_time / T_max                    # (R, 64)
feat_energy = np.sum(np.square(feat_seq), axis=1)             # (R, 64)

# Stack features horizontally
X_features = np.hstack(
    [
        feat_mean,
        feat_std,
        feat_max,
        feat_mean_abs_diff,
        feat_std_accel,
        feat_mean_half1,
        feat_mean_half2,
        feat_zcr,
        feat_max_time_norm,
        feat_energy,
    ]
)
# ... (Previous feature extraction and splitting code remains the same) ...

print(f"Final feature array shape: {X_features.shape}")  # (R, 640)




# ---------------------------------------------------
# Train/test split + scaling
# ---------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X_features,
    gnd_seq,
    test_size=0.2,
    random_state=42,
    stratify=gnd_seq,
)

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")

Loaded CoST shape: (1496855, 68)
Max frame length: 1747
Number of sequence starts: 7805
Number of sequences after split: 7805
Final tensor shape: R=7805, T_max=1747, C=64
Final feature array shape: (7805, 640)
Train size: 6244, Test size: 1561


In [ ]:


# ---------------------------------------------------
# Fast sanity check: single EBM fit
# ---------------------------------------------------

# OPTIMIZED SETTINGS FOR SPEED
ebm_fast = ExplainableBoostingClassifier(
    max_rounds=50,       # Reduced slightly for speed
    outer_bags=4,
    inner_bags=0,
    learning_rate=0.05,
    max_bins=32,
    interactions=0,      # <--- CRITICAL: Disables heavy pair-detection (Major speedup)
    n_jobs=-1,           # <--- CRITICAL: Uses all CPU cores
    random_state=42,
)

print("Fitting single EBM (sanity check)...")
ebm_fast.fit(X_train, y_train)
y_pred = ebm_fast.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy (single EBM, no CV): {test_acc:.3f}")

# ---------------------------------------------------
# Cross-validation with lighter settings
# ---------------------------------------------------

param_grid_ebc = {
    "max_bins": [16, 32, 64],
    "learning_rate": [0.01, 0.05],
    "interactions": [0, 5],  # Keep interactions at 0
    "max_rounds": [50, 100, 500],
    "outer_bags": [2, 4, 8],
    "inner_bags": [0, 4, 8],
}

def cross_validate_model_light(model, train, param_grid, cv_folds):
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv_folds,
        verbose=3,
        n_jobs=1,  # Keep this 1, because EBM handles the parallelism internally now
    )
    grid_search.fit(train["feat"], train["gnd"])
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print(f"[BEST] Params: {best_params} -> Average Accuracy: {best_score}")
    return pd.DataFrame(grid_search.cv_results_), best_params


print("Running cross-validation EBM (Optimized)...")
df_cost, best_params_ebc = cross_validate_model_light(
    ExplainableBoostingClassifier(
        max_rounds=50,       # Keep rounds low for CV
        outer_bags=4,
        inner_bags=0,
        learning_rate=0.05,
        max_bins=32,
        interactions=0,      # <--- CRITICAL FIX
        n_jobs=-1,           # <--- Uses all cores
        random_state=42,
    ),
    {"feat": X_train, "gnd": y_train},
    param_grid_ebc,
    cv_folds=3,  # 3 folds is faster than 5
)

print("Best params for ExplainableBoostingClassifier on CoST:", best_params_ebc)

Fitting single EBM (sanity check)...
Test accuracy (single EBM, no CV): 0.642
Running cross-validation EBM (Optimized)...
Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV 1/3] END inner_bags=0, interactions=0, learning_rate=0.01, max_bins=16, max_rounds=50, outer_bags=2;, score=0.622 total time=   7.3s
[CV 2/3] END inner_bags=0, interactions=0, learning_rate=0.01, max_bins=16, max_rounds=50, outer_bags=2;, score=0.608 total time=   7.4s
[CV 3/3] END inner_bags=0, interactions=0, learning_rate=0.01, max_bins=16, max_rounds=50, outer_bags=2;, score=0.612 total time=   7.2s
[CV 1/3] END inner_bags=0, interactions=0, learning_rate=0.01, max_bins=16, max_rounds=50, outer_bags=4;, score=0.620 total time=   7.5s
[CV 2/3] END inner_bags=0, interactions=0, learning_rate=0.01, max_bins=16, max_rounds=50, outer_bags=4;, score=0.612 total time=   7.6s
[CV 3/3] END inner_bags=0, interactions=0, learning_rate=0.01, max_bins=16, max_rounds=50, outer_bags=4;, score=0.614 total time= 

In [ ]:
best_model = ExplainableBoostingRegressor(
    max_rounds=best_params_ebc.get("max_rounds", 50),       # Keep rounds low for CV
    outer_bags=best_params_ebc.get("outer_bags", 4),
    inner_bags=best_params_ebc.get("inner_bags", 0),
    learning_rate=best_params_ebc.get("learning_rate", 0.05),
    max_bins=best_params_ebc.get("max_bins", 32),
    interactions=best_params_ebc.get("interactions", 0),      # <--- CRITICAL FIX
    n_jobs=-1,           # <--- Uses all cores
    random_state=42
)

best_model.fit(X_train, y_train)
print("Trained final EBM model with best hyperparameters.")

y_pred = best_model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy (final EBM): {test_acc:.3f}")
    